In [2]:
import pandas as pd

# read the entire file into a python array
with open('yelp_academic_dataset_business.json', 'rb') as f:
    data = f.readlines()
    
print data[0:2]

['{"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\\nSte 101\\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 9, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}\n', '{"business_id": "UsFtqoBl7naz8AVUBZMjQQ", "full_address": "202 McClure St\\nDravosburg, PA 15034", "hours": {}, "open": true, "categories": ["Nightlife"], "city": "Dravosburg", "review_count": 4, "name": "Clancy\'s Pub", "neighborhoods": [], "longitude": -79.886930000000007, "state": "PA", "stars": 3.5, "latitude": 

In [3]:
# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

print data[:2]

['{"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\\nSte 101\\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 9, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}', '{"business_id": "UsFtqoBl7naz8AVUBZMjQQ", "full_address": "202 McClure St\\nDravosburg, PA 15034", "hours": {}, "open": true, "categories": ["Nightlife"], "city": "Dravosburg", "review_count": 4, "name": "Clancy\'s Pub", "neighborhoods": [], "longitude": -79.886930000000007, "state": "PA", "stars": 3.5, "latitude": 40

In [4]:
# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
df = pd.read_json(data_json_str)

df[:2]

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{u'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{u'Tuesday': {u'close': u'17:00', u'open': u'0...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,9,3.5,AZ,business
1,"{u'Happy Hour': True, u'Accepts Credit Cards':...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350519,-79.886930,Clancy's Pub,[],True,4,3.5,PA,business


In [8]:
len(df.state)

61184

In [11]:
#There's an issue with German reviews being in German... so let's filter out everything 
#except USA and Ontario. (Edinburgh should be ok, but let's cut in case review writing style differs.)

df.state.unique()

array([u'AZ', u'PA', u'NC', u'SC', u'WI', u'IL', u'NV', u'WA', u'CA',
       u'QC', u'ON', u'EDH', u'MLN', u'ELN', u'HAM', u'SCB', u'FIF',
       u'XGL', u'NTH', u'BW', u'RP', u'KHL', u'MN', u'MA', u'NW', u'OR'], dtype=object)

In [12]:
skipState = ['QC', 'EDH', 'MLN', 'ELN', 'HAM', 'SCB', 'FIF', 'XGL', 'NTH', 'BW', 'RP', 'KHL', 'NW']
keepState = ['AZ', 'PA', 'NC', 'SC', 'WI', 'IL', 'NV', 'WA', 'CA', 'ON', 'MN', 'MA', 'OR']
print len(skipState), len(keepState), len(df.state.unique())

13 13 26


In [27]:
print df.shape
#print df[df.state in keepState].shape
a = len(df[df.state.map(lambda x: x in keepState)])
b = len(df[df.state.map(lambda x: x in skipState)])
print a,b,a+b, (b*1.0)/(a+b)

(61184, 15)
53200 7984 61184 0.130491631799


Awesome, so filtering out these things outside USA/Ontario, we only lose 13% of reviews.

In [29]:
#Restrict to USA+ON
df = df[df.state.map(lambda x: x in keepState)]
df.shape

(53200, 15)

In [31]:
#Restrict to Restaurants.
df = df[df.categories.map(lambda x: "Restaurants" in x)]
df.shape

(17914, 15)

Save USA+ON Restaurants to a file!

In [36]:
#I want to save this as a file, but I get errors.  The first error is complaining 
# about an accented e in the data... 

df.to_csv("yelp_business.csv", encoding='utf-8')

This file above is useful for all restaurants.  We also need a file that can tell us just the restaurants with a lot of reviews.

In [40]:
#Restrict to restaurants with >=100 reviews.
restaurantsMR = df[df.review_count>=100]
print "We're continuing with %d restaurant, with 100+ reviews." %len(restaurantsMR)

We're continuing with 2595 restaurant, with 100+ reviews.


In [41]:
#Make a smaller DataFrame with 3 attributes: BusinessID, NumRevs, Stars

keyattrib = restaurantsMR[["business_id","review_count", "stars"]]

print keyattrib.shape
keyattrib.head()

(2595, 3)


,business_id,review_count,stars
136,2SwC8wqpZC4B9iFVTgYT9A,199,4.0
137,PzG0XmAmtXFs6Fdnubf41g,143,4.0
166,IxQ1ATP_Wg_QujO9nywzcQ,110,4.0
237,2Igiyhi82R8_QaaPplRmCw,173,4.5
255,2X5G4Ujq0s4Wfn4TC7gX0g,225,4.5


In [42]:
print "Number of these restaurants with >4 stars: ", len(keyattrib[keyattrib.stars>4])
print "Number of these restaurants with <3 stars: ", len(keyattrib[keyattrib.stars<3])


Number of these restaurants with >4 stars:  312
Number of these restaurants with <3 stars:  70


In [43]:
#Save three files for 'keyattribs' info on all, good, and bad restaurants with >100 reviews.
keyattrib.to_csv("restos100plusrevsALL.csv")
keyattrib[keyattrib.stars>4].to_csv("restos100plusrevsGOOD.csv")
keyattrib[keyattrib.stars<3].to_csv("restos100plusrevsBAD.csv")